In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 1. 读入初步处理好的数据
df_data = pd.read_csv('初步处理data_全部.csv')
df_data.head() # 查看头五行

,Unnamed: 0,Anon Student Id,Student Response Subtype,Duration (sec),Outcome
0,0,Stu_11eed491c49d08923078a99c3689faa6,DELETION,0,NaN
1,1,Stu_11eed491c49d08923078a99c3689faa6,Axes,17.5,CORRECT
2,2,Stu_11eed491c49d08923078a99c3689faa6,DELETION,17.5,NaN
3,3,Stu_11eed491c49d08923078a99c3689faa6,NextStepHelp,74,HINT
4,4,Stu_11eed491c49d08923078a99c3689faa6,DELETION,0.5,NaN


In [3]:
# 2. 打印一个学生的数据观察
single_stu_df = df_data[df_data['Anon Student Id'] == 'Stu_11eed491c49d08923078a99c3689faa6']
single_stu_df.groupby("Student Response Subtype").agg('sum')

,Unnamed: 0
Student Response Subtype,
Answer,135810
Axes,68385
Body,91332
Choose-,18997
Choose-Equation,2594
Choose-Principle,108784
Choose-Sought,126621
DELETION,111265
Equation,524816


In [4]:
# 3. 查看有多少学生
stu_count_df = df_data.drop_duplicates(subset = ['Anon Student Id', ], keep = 'first', inplace=False)
print("总共有{}个学生。".format(len(stu_count_df)))
stu_count_df

总共有69个学生。


,Unnamed: 0,Anon Student Id,Student Response Subtype,Duration (sec),Outcome
0,0,Stu_11eed491c49d08923078a99c3689faa6,DELETION,0,NaN
3384,3400,Stu_16d36e81d1d34630d27d8fd8b9d99d53,NextStepHelp,8,HINT
8233,8266,Stu_1712d2d13ed773fddf0c9a59ad099b21,Axes,73,INCORRECT
8267,8300,Stu_18d58be83dc070054ca65038cc2293b4,Body,4.75,CORRECT
9869,9906,Stu_1a24bc4f1db0936c28f45176d3c425a7,Body,12.5,CORRECT
...,...,...,...,...,...
158456,159329,Stu_f8258e6b33109ea4808c004915145dd2,Answer,18,INCORRECT
162063,162961,Stu_fa1d7f2f99f036137482fbc04bab625b,NextStepHelp,46,HINT
165130,166031,Stu_fb6121b7f70cc8dc2dca6a97e2c7cdc5,Vector,84,INCORRECT
169221,170137,Stu_fd75420a601b7a43d13ea33ce657a2e1,DELETION,0,NaN


**4. 接下来将计算每个学生的每个行为的持续性时间，每个行为的频次，正确率**
<br>

|  行为 | Behavior  |指标|
|  ----  | ----  | ----  |
|绘制/生成坐标|Axes|（时间，频次，正确率）|
|绘制向量|Vector|（时间，频次，正确率）|
|定义变量|Variable|（频次，正确率）|
|公式推导|Equation|（时间，频次，正确率）|
|查看资料|Choose-Principle|（频次）|
|回答问题|Answer|（时间，频次，正确率）|
|错题订正|Whats-Wrong|（频次）|
|尝试提交|Solve|（频次）|

<br><br>
事先说明一下，最后处理出来的每个学生的信息是一个八个维度的向量：
<br>
stu_demo = (Axes, Vector, Variable, Equation, Choose-Principle, Answer, Whats-Wrong, Solve)
---
频次版本字段要求：
stu_demo = (Axes, Vector, Variable, Equation, Choose-Principle, Answer, Whats-Wrong, Solve, Explain-Further, NextStepHelp, Choose-Sought)
--- 
01.26 频次版本字段要求：
stu_demo = (Axes, Vector, Variable, Equation, Choose-Principle, Answer, Whats-Wrong, Solve, Explain-Further, NextStepHelp)

In [5]:
# 4.1 首先，我们提取一个包含所有学生的列表；因为接下来要分别对每个学生统计以上数据。
temp_li = [] # 暂时用来存每个学生的DataFrame
stu_list=stu_count_df['Anon Student Id'].values.tolist()
for stu,i in zip(stu_list, range(len(stu_list))):
    single_stu_df = df_data[df_data['Anon Student Id'] == stu]
    # 统计频次
    count_srs = single_stu_df.groupby("Student Response Subtype").agg('count')
    
    # 然后将每种学习行为和频次用键值对（字典）一一对应起来，顺便把学生的 ID：‘@#￥%@’ 这样的加入字典
    # 学习行为(因为有些学生的某些学习行为是没有的，所以需要单独提取出来)
    stu_study_behav = list(count_srs.index)
    stu_study_behav.insert(0, 'Anon Student Id')
    
    # 学习行为对应的频次(加入学生ID)
    stu_behavior_freq = list(count_srs['Anon Student Id'])
    stu_behavior_freq.insert(0, stu)
    
    # 将“学习行为字段”和“学习行为频次”组成字典
    single_stu_temp_dict = dict(zip(stu_study_behav, stu_behavior_freq))
    
    # 然后根据字典生成当前学生的DataFrame
    single_stu_df_processed = pd.DataFrame(single_stu_temp_dict, index = [0])
    temp_li.append(single_stu_df_processed)

In [6]:
# 将所有学生的学习行为频次组合起来
all_stu_behav = pd.concat(temp_li)
# 将缺失值补充为0
all_stu_behav = all_stu_behav.fillna(0)
# 保存为CSV
# all_stu_behav.to_csv("学生频次数据_0126.csv")

all_stu_behav

,Anon Student Id,Answer,Axes,Body,Choose-,Choose-Equation,Choose-Principle,Choose-Sought,DELETION,Equation,Explain-Further,NextStepHelp,Simplify,Solve,Variable,Vector,Whats-Wrong
0,Stu_11eed491c49d08923078a99c3689faa6,82.0,51.0,61.0,13.0,1.0,62.0,77.0,87,312.0,1421.0,505.0,1.0,146.0,147.0,349.0,69.0
0,Stu_16d36e81d1d34630d27d8fd8b9d99d53,100.0,57.0,70.0,19.0,0.0,77.0,87.0,120,437.0,2138.0,818.0,0.0,162.0,212.0,515.0,37.0
0,Stu_1712d2d13ed773fddf0c9a59ad099b21,0.0,3.0,2.0,0.0,0.0,0.0,0.0,3,0.0,7.0,1.0,0.0,0.0,0.0,14.0,4.0
0,Stu_18d58be83dc070054ca65038cc2293b4,69.0,34.0,32.0,9.0,0.0,33.0,31.0,38,157.0,642.0,247.0,0.0,52.0,68.0,156.0,34.0
0,Stu_1a24bc4f1db0936c28f45176d3c425a7,15.0,40.0,56.0,1.0,0.0,46.0,42.0,91,463.0,436.0,247.0,0.0,58.0,160.0,231.0,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Stu_f8258e6b33109ea4808c004915145dd2,83.0,58.0,94.0,12.0,2.0,63.0,79.0,184,1018.0,696.0,359.0,2.0,226.0,227.0,442.0,62.0
0,Stu_fa1d7f2f99f036137482fbc04bab625b,209.0,39.0,63.0,14.0,0.0,51.0,45.0,58,288.0,1299.0,455.0,0.0,127.0,102.0,267.0,50.0
0,Stu_fb6121b7f70cc8dc2dca6a97e2c7cdc5,101.0,56.0,79.0,6.0,0.0,75.0,77.0,120,1011.0,1158.0,552.0,1.0,207.0,238.0,351.0,59.0
0,Stu_fd75420a601b7a43d13ea33ce657a2e1,112.0,4.0,11.0,0.0,0.0,0.0,0.0,16,5.0,19.0,3.0,0.0,0.0,6.0,33.0,22.0


17

In [11]:
# 4.2 接下来进行数据归一化：

# 归一化函数
max_min_scaler = lambda x : (x - np.min(x)) / (np.max(x) - np.min(x))

# 从表中左侧学生行为的第一列开始到最后一列执行归一化操作
temp_df_behav = all_stu_behav.iloc[:, [x for x in range(1, int(all_stu_behav.shape[1]))]].apply(max_min_scaler)
all_stu_behav_processed = pd.concat([all_stu_behav[['Anon Student Id']], temp_df_behav], axis = 1)
all_stu_behav_processed.to_csv('学生频次数据_归一化_全部行为数据.csv', encoding = 'utf8')
all_stu_behav_processed

,Anon Student Id,Answer,Axes,Body,Choose-,Choose-Equation,Choose-Principle,Choose-Sought,DELETION,Equation,Explain-Further,NextStepHelp,Simplify,Solve,Variable,Vector,Whats-Wrong
0,Stu_11eed491c49d08923078a99c3689faa6,0.344538,0.579545,0.488,0.419355,0.025641,0.548673,0.620968,0.251497,0.215470,0.664640,0.607702,0.010638,0.536765,0.398374,0.343166,0.267442
0,Stu_16d36e81d1d34630d27d8fd8b9d99d53,0.420168,0.647727,0.560,0.612903,0.000000,0.681416,0.701613,0.350299,0.301796,1.000000,0.984356,0.000000,0.595588,0.574526,0.506391,0.143411
0,Stu_1712d2d13ed773fddf0c9a59ad099b21,0.000000,0.034091,0.016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003274,0.001203,0.000000,0.000000,0.000000,0.013766,0.015504
0,Stu_18d58be83dc070054ca65038cc2293b4,0.289916,0.386364,0.256,0.290323,0.000000,0.292035,0.250000,0.104790,0.108425,0.300281,0.297232,0.000000,0.191176,0.184282,0.153392,0.131783
0,Stu_1a24bc4f1db0936c28f45176d3c425a7,0.063025,0.454545,0.448,0.032258,0.000000,0.407080,0.338710,0.263473,0.319751,0.203929,0.297232,0.000000,0.213235,0.433604,0.227139,0.356589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Stu_f8258e6b33109ea4808c004915145dd2,0.348739,0.659091,0.752,0.387097,0.051282,0.557522,0.637097,0.541916,0.703039,0.325538,0.432010,0.021277,0.830882,0.615176,0.434612,0.240310
0,Stu_fa1d7f2f99f036137482fbc04bab625b,0.878151,0.443182,0.504,0.451613,0.000000,0.451327,0.362903,0.164671,0.198895,0.607577,0.547533,0.000000,0.466912,0.276423,0.262537,0.193798
0,Stu_fb6121b7f70cc8dc2dca6a97e2c7cdc5,0.424370,0.636364,0.632,0.193548,0.000000,0.663717,0.620968,0.350299,0.698204,0.541628,0.664260,0.010638,0.761029,0.644986,0.345133,0.228682
0,Stu_fd75420a601b7a43d13ea33ce657a2e1,0.470588,0.045455,0.088,0.000000,0.000000,0.000000,0.000000,0.038922,0.003453,0.008887,0.003610,0.000000,0.000000,0.016260,0.032448,0.085271
